# Защита персональных данных клиентов

## Загрузка данных

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# чтение файла с данными и сохранение в df с использованием конструкции try-except
try:
    df = pd.read_csv('C:/Users/datasets/insurance.csv') # локальный путь
except:
    df = pd.read_csv('//datasets/insurance.csv') # серверный путь

In [3]:
# просмотр первых пяти строк 
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# получение общей информации
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:** 

$$
a_1 = X P w
$$
$$
a_1 = X P((XP)^T XP)^{-1} (XP)^T y
$$
$$
a_1 = (X P) (XP)^{-1}((XP)^T)^{-1} (XP)^T y
$$
$$
a_1 = X P P^{-1}X^{-1}(P^T X^T)^{-1} P^T X^T y = XEX^{-1}(X^T)^{-1}(P^T)^{-1}P^T X^T y
$$
$$
a_1 = XX^{-1}(X^T)^{-1}EX^T y = X(X^T X)^{-1} X^T y = X w = a
$$

## Алгоритм преобразования

**Алгоритм**

Чтобы защитить данные клиентов, будем умножать матрицу признаков на обратимую матрицу, сгенерированную случайным образом. Таким образом, алгоритм преобразования данных будет следущим: составление обратимой матрицы; проверка матрицы на обратимость; получение матрицы преобразованных признаков; проверка качества линейной регресси на преобразованных данных.

**Обоснование**

Пример:
$$
X = \begin{pmatrix}
     2& 4\\
     3& 7\\
     5& 1\
    \end{pmatrix}
$$
$$
P = \begin{pmatrix}
     1& 2\\
     0& 3\
    \end{pmatrix}
$$
$$
Z = XP = \begin{pmatrix}
          2*1+4*0& 2*2+4*3\\
          3*1+7*0& 3*2+7*3\\
          5*1+1*0& 5*2+1*3\
         \end{pmatrix} = \begin{pmatrix}
                          2& 16\\
                          3& 27\\
                          5& 13\
                         \end{pmatrix}           
$$

При этом чтобы *Z* матрица имела такую же размерность, что и матрица *Х*. А для этого матрица *Р* должна быть квадратной (у которой число строк и столбцов равно количеству признаков.

## Проверка алгоритма

In [5]:
# создание переменных с признакими и целевым признаком
target = df['Страховые выплаты']
features = df.drop('Страховые выплаты', axis=1)

print(target.shape)
features.shape

(5000,)


(5000, 4)

In [6]:
# функция для расчета метрики r2
def lin_regression(x, y):
    model = LinearRegression()
    model.fit(x, y)
    return r2_score(y, model.predict(x))

In [7]:
# расчет r2
lin_regression(features, target)

0.42494550286668

In [8]:
# создание обратимой матрицы
P = np.random.normal(size=(4, 4))
print(P)

np.linalg.inv(P) # проверка обратимости матрицы

[[-0.33525568  0.42902486 -0.65025013 -0.56655532]
 [-0.12096238 -0.50259556 -0.82363436  0.15832097]
 [-0.69055439  1.41971986  2.1411035  -1.436361  ]
 [-0.95048715  1.80042386 -0.06471185  0.79497353]]


array([[ 0.90654163, -2.97839199, -0.88106454, -0.35268953],
       [ 0.80398287, -1.6517513 , -0.38498706,  0.20633045],
       [-0.77756689,  0.26999911,  0.33466749, -0.00324268],
       [-0.80024244,  0.20176933, -0.15427392,  0.36866765]])

In [10]:
# получение матрицы преобразованных признаков
Z = features @ P
Z.head()

,0,1,2,3
0,-34257.743024,70399.728080,106164.249485,-71236.786253
1,-26247.581637,53928.035703,81323.980997,-54573.640438
2,-14505.150132,29799.541786,44939.288043,-30158.989789
3,-28800.559314,59195.364497,89266.590084,-59891.339206
4,-18027.191823,37041.044692,55859.089262,-37485.155789


In [11]:
# проверка качества линейной регрессии
lin_regression(Z, target)

0.42494550286667043

Таким образом, качество линейной регрессии не меняется при умножении матрицы признаков на квадратную обратимую матрицу.